In [1]:
model_name = 'sparse_tfidf'
data_dir = './data/mlboot_dataset/'
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
import lightgbm as lgb
import scipy.sparse as sp
import gc

In [2]:
from sklearn.feature_extraction.text import TfidfTransformer
df = pd.read_csv(data_dir + 'preprocessed.csv') 
q = pd.read_csv(data_dir + 'sessions.csv')
df = df.merge(q, on='uid', how='left')
del q
y = pd.read_table(data_dir + 'mlboot_train_answers.tsv')
y.columns = ['uid','target']
df = df.merge(y, on='uid', how='left')

df_train_index = df[~df.target.isnull()].index
df_test_index = df[df.target.isnull()].index

mat = sp.load_npz(data_dir+'dmat1.npz').tolil()
#mat2 = sp.load_npz(data_dir+'dmat2.npz').tolil()
#mat3 = sp.load_npz(data_dir+'dmat3.npz').tolil()

#mat = sp.hstack([mat1,mat2,mat3]).tolil()
#del mat1,mat2,mat3

train_mat = mat[df_train_index.tolist()]
test_mat = mat[df_test_index.tolist()]
mat = mat.tocsc()[:, np.where((train_mat.getnnz(axis=0) > 0) & (test_mat.getnnz(axis=0) > 0))[0]].tocsr()

import tqdm
idcs = []
for q in tqdm.tqdm(range(len(df))):
    idcs.append(mat[q].nonzero()[1] + 1)
df['idcs'] = np.array(idcs)
del idcs
gc.collect()

100%|██████████| 609018/609018 [01:15<00:00, 8047.10it/s]


7

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
del mat
gc.collect()

0

In [4]:
df['idcs'] = df.idcs.apply(lambda x : ' '.join([str(q) for q in x.tolist()]))
mat = TfidfVectorizer(max_features=5000).fit_transform(df.idcs)

In [5]:
X = df.loc[~df.target.isnull(),:].reset_index(drop=True)
x_te = df.loc[df.target.isnull(),:].reset_index(drop=True)

In [6]:
%%time
train_mat = mat[df_train_index.tolist()]
test_mat = mat[df_test_index.tolist()]

CPU times: user 2.53 s, sys: 140 ms, total: 2.67 s
Wall time: 2.14 s


In [18]:
import xgboost as xgb
import scipy.sparse as sp
from sklearn.feature_selection import SelectPercentile
from sklearn.preprocessing import minmax_scale
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler
from vowpalwabbit.sklearn_vw import VWClassifier,VWRegressor
from sklearn.linear_model import SGDRegressor,SGDClassifier,Ridge

def make_agg_features(X, train_index, test_index, test_data):
    return X.loc[train_index,:], X.loc[test_index,:], test_data
    
train_cols = [
       'sess_keys_mean','sess_keys_max','diff_key1_mean','diff_key1_max','diff_key2_mean',
       'diff_key2_max','diff_key3_mean','diff_key3_max','quot_key1_mean','quot_key1_max',
       'quot_key2_mean','quot_key2_max','quot_key3_mean','quot_key3_max',
       u'num_keys_f1',
       u'sum_values_f1', u'num_keys_f2', u'sum_values_f2', u'num_keys_f3',
       u'sum_values_f3', u'num_times_cat_eq_0', u'num_times_cat_eq_1',
       u'num_times_cat_eq_2', u'num_times_cat_eq_3', u'num_times_cat_eq_4',
       u'num_times_cat_eq_5', u'records', u'max_days', u'min_days',
       u'sum_values_f1_std', u'num_keys_f1_std', u'sum_values_f2_std',
       u'num_keys_f2_std', u'sum_values_f3_std', u'num_keys_f3_std',
       u'sum_values_f1_max', u'num_keys_f1_max', u'sum_values_f2_max',
       u'num_keys_f2_max', u'sum_values_f3_max', u'num_keys_f3_max',
       u'sum_values_f1_mean', u'num_keys_f1_mean', u'sum_values_f2_mean',
       u'num_keys_f2_mean', u'sum_values_f3_mean', u'num_keys_f3_mean'] 
    
kf = KFold(n_splits=10, shuffle=True, random_state=239)

ifold = 0

y_pred = 0
y_oof = X[['uid','target']].copy()
y_oof['target'] = np.nan

scores = []

for train_index,test_index in kf.split(X):
    print('fold', ifold)
       
    y_tr,y_va = X.loc[train_index,'target'].values,X.loc[test_index,'target'].values
    
    X_tr,X_va,X_te = make_agg_features(X,train_index,test_index,x_te)
    X_tr = X_tr[train_cols].fillna(0)
    X_va = X_va[train_cols].fillna(0)
    X_te = X_te[train_cols].fillna(0)
    
    scaler = MinMaxScaler(feature_range=(-1,1))
    X_tr = scaler.fit_transform(X_tr)
    X_va = scaler.transform(X_va)
    X_te = scaler.transform(X_te)
    
    yy = y_tr
    ssp = SelectPercentile(percentile=20)  
    ssp.fit(train_mat[train_index], yy)
    sp_train_mat = ssp.transform(train_mat[train_index])
    sp_val_mat = ssp.transform(train_mat[test_index])
    sp_test_mat = ssp.transform(test_mat)   
    
    print('prepare train')
    X_tr = sp.hstack([
        X_tr, sp_train_mat
    ]).tocsr()
    print(X_tr.shape)
    print('prepare valid')
    X_va = sp.hstack([
        X_va, sp_val_mat
    ]).tocsr()    
    print('prepare test')
    X_te = sp.hstack([
        X_te, sp_test_mat
    ]).tocsr()     
    
    model = Ridge()
    model.fit(X_tr,y_tr)
    
    yhat = model.predict(X_va)
    scores.append(roc_auc_score(y_va,yhat))
    print(ifold,roc_auc_score(y_va,yhat))
    y_oof.loc[test_index,'target'] = yhat
   
    ytst = model.predict(X_te)
    print(ytst)
    print(minmax_scale(ytst))
    y_pred += minmax_scale(ytst)*0.1
    
    del X_tr,X_va,X_te, sp_train_mat, sp_val_mat, sp_test_mat
    gc.collect()    
    
    ifold += 1 
#model = VWRegressor(power_t=0.99, ftrl=True, l1=15, l2=0.1)
#0.678657863271
#0.654927206221

#0.8 perc
# 0 0.681140205086
# 1 0.664159676552
# 2 0.661224538511
# 3 0.671735521827

fold 0
prepare train
(385194, 1047)
prepare valid
prepare test
0 0.667640045102
[ 0.07349721  0.12928352  0.05150488 ...,  0.07837062  0.06115719
  0.02823173]
[ 0.40577469  0.45913654  0.38473815 ...,  0.41043631  0.39397097
  0.36247645]
fold 1


/root/miniconda3/lib/python3.6/site-packages/sklearn/feature_selection/univariate_selection.py:113: UserWarning: Features [0] are constant.
  UserWarning)
/root/miniconda3/lib/python3.6/site-packages/sklearn/feature_selection/univariate_selection.py:114: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


prepare train
(385194, 1047)
prepare valid
prepare test
1 0.659310694371
[ 0.06992129  0.12683005  0.05159513 ...,  0.0388825   0.07637881
  0.03104474]
[ 0.39352646  0.44440091  0.37714351 ...,  0.36577887  0.39929926
  0.35877218]
fold 2
prepare train
(385194, 1047)
prepare valid
prepare test
2 0.656572926728
[ 0.07236681  0.14677482  0.05314057 ...,  0.08369351  0.07338431
  0.0324171 ]
[ 0.38279577  0.45369657  0.36447576 ...,  0.39358859  0.38376531
  0.34472909]
fold 3
prepare train
(385194, 1047)
prepare valid
prepare test
3 0.668256747525
[ 0.07156164  0.12614479  0.0488369  ...,  0.03938583  0.07105935
  0.00964939]
[ 0.41778379  0.46814681  0.39681602 ...,  0.38809567  0.41732033
  0.36065833]
fold 4
prepare train
(385195, 1047)
prepare valid
prepare test
4 0.66267845102
[ 0.07003006  0.13374442  0.04710925 ...,  0.03885763  0.06714329
  0.02437874]
[ 0.28232381  0.34860959  0.25847794 ...,  0.2498933   0.27932053
  0.23483007]
fold 5
prepare train
(385195, 1047)
prepare vali

In [19]:
model_name = 'tfidfr_5folds_1'

print(scores,np.mean(scores))
print(roc_auc_score(X.target.values, y_oof.target.values))

results_dir = './results/'
np.save(results_dir + 'train_' + model_name +'.npy', y_oof.target.values)
sample_sub = pd.read_table(data_dir+'mlboot_test.tsv')

sub = x_te[['uid','target']].copy()
sub['target'] = y_pred
sub.columns = ['cuid','target']
sample_sub = sample_sub.merge(sub, on='cuid', how='left')
np.save(results_dir + 'test_' + model_name +'.npy', sample_sub.target.values)
print('isnull?',sample_sub.target.isnull().any())
sample_sub[['target']].to_csv(results_dir + model_name + '.csv', header=False, index=False)
sample_sub.head()

[0.66764004510150232, 0.65931069437122747, 0.65657292672814449, 0.66825674752497444, 0.66267845102005618, 0.6658238069261444, 0.6661894787624616, 0.67047324091370886, 0.66749289183146332, 0.66964968219749332] 0.665408796538
0.6653782259
isnull? False


,cuid,target
0,888b238b4d14c03173baa375a739f6bc,0.567283
1,ac4b8244f3ae82df511b002257473c11,0.375874
2,483d8b91e49522c8a5bbe37f3872c749,0.436236
3,4c7ec46a0e88a7e1e1cedd2d526d5d61,0.363009
4,fdbfba9842ff0bf86d600eb334c7c42b,0.351171


In [16]:
mat = TfidfVectorizer().fit_transform(df.idcs)
from sklearn.decomposition import TruncatedSVD
pca = TruncatedSVD(n_components = 300, algorithm='arpack')
mat = pca.fit_transform(mat.astype(np.float32))
train_mat = mat[df_train_index.tolist()]
test_mat = mat[df_test_index.tolist()]

In [17]:
np.save(data_dir+'svd_tfidf300.npy',mat)

In [ ]:
train_cols = [
       'sess_keys_mean','sess_keys_max','diff_key1_mean','diff_key1_max','diff_key2_mean',
       'diff_key2_max','diff_key3_mean','diff_key3_max','quot_key1_mean','quot_key1_max',
       'quot_key2_mean','quot_key2_max','quot_key3_mean','quot_key3_max',
       u'num_keys_f1',
       u'sum_values_f1', u'num_keys_f2', u'sum_values_f2', u'num_keys_f3',
       u'sum_values_f3', u'num_times_cat_eq_0', u'num_times_cat_eq_1',
       u'num_times_cat_eq_2', u'num_times_cat_eq_3', u'num_times_cat_eq_4',
       u'num_times_cat_eq_5', u'records', u'max_days', u'min_days',
       u'sum_values_f1_std', u'num_keys_f1_std', u'sum_values_f2_std',
       u'num_keys_f2_std', u'sum_values_f3_std', u'num_keys_f3_std',
       u'sum_values_f1_max', u'num_keys_f1_max', u'sum_values_f2_max',
       u'num_keys_f2_max', u'sum_values_f3_max', u'num_keys_f3_max',
       u'sum_values_f1_mean', u'num_keys_f1_mean', u'sum_values_f2_mean',
       u'num_keys_f2_mean', u'sum_values_f3_mean', u'num_keys_f3_mean']

# Train the model
parameters = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'auc',
    'num_leaves': 128,
    #'max_depth' : 12,
    #'min_data' : 30,
    'lambda_l2' : 41.5,
    #'min_sum_hessian_in_leaf' : 0.3,
    'max_bin': 64,
    #'max_drop' : 1,
    'learning_rate': 0.01,
    'feature_fraction': 0.27,
    'verbose': 0
}


kf = KFold(n_splits=5, shuffle=True, random_state=239)

ifold = 0

y_pred = 0
y_oof = X[['uid','target']].copy()
y_oof['target'] = np.nan

scores = []

for train_index,test_index in kf.split(X):       
    print('fold', ifold)
    
    y_tr,y_va = X.loc[train_index,'target'].values,X.loc[test_index,'target'].values
    X_tr,X_va = X.loc[train_index, train_cols].values,X.loc[test_index, train_cols].values
    X_te = x_te[train_cols]
    
    print('prepare train')
    X_tr = np.hstack([
        X_tr, train_mat[train_index]
    ])
    print('prepare valid')
    X_va = np.hstack([
        X_va, train_mat[test_index]
    ])
    
    # Create the LightGBM data containers
    tr_data = lgb.Dataset(X_tr, label=y_tr) #, categorical_feature=cate_cols
    va_data = lgb.Dataset(X_va, label=y_va) #, categorical_feature=cate_cols

    model = lgb.train(parameters,
                      tr_data,
                      valid_sets=va_data,
                      num_boost_round=8000,
                      early_stopping_rounds=200,
                      verbose_eval=50)
    
    yhat = model.predict(X_va, model.best_iteration)
    print(ifold,roc_auc_score(y_va,yhat))
    scores.append(roc_auc_score(y_va,yhat))
    y_oof.loc[test_index,'target'] = yhat

    print('prepare test')
    X_te = np.hstack([
        X_te, test_mat
    ])
    
    ytst = model.predict(X_te, model.best_iteration)
    y_pred += ytst*0.1
    
    ifold += 1

fold 0
prepare train
prepare valid
Training until validation scores don't improve for 200 rounds.
[50]	valid_0's auc: 0.609881
[100]	valid_0's auc: 0.626785
[150]	valid_0's auc: 0.636711
[200]	valid_0's auc: 0.645291
[250]	valid_0's auc: 0.653108
[300]	valid_0's auc: 0.659199
[350]	valid_0's auc: 0.66384
[400]	valid_0's auc: 0.667295
[450]	valid_0's auc: 0.670719
[500]	valid_0's auc: 0.67305
[550]	valid_0's auc: 0.674532
[600]	valid_0's auc: 0.676042
[650]	valid_0's auc: 0.677376
[700]	valid_0's auc: 0.678354
[750]	valid_0's auc: 0.679278
[800]	valid_0's auc: 0.679786
[850]	valid_0's auc: 0.680553
[900]	valid_0's auc: 0.681362
[950]	valid_0's auc: 0.681892
[1000]	valid_0's auc: 0.68238
[1050]	valid_0's auc: 0.682756
[1100]	valid_0's auc: 0.683065
[1150]	valid_0's auc: 0.68333
[1200]	valid_0's auc: 0.683592
[1250]	valid_0's auc: 0.683563
[1300]	valid_0's auc: 0.683724
[1350]	valid_0's auc: 0.683775
[1400]	valid_0's auc: 0.683777
[1450]	valid_0's auc: 0.683716
[1500]	valid_0's auc: 0.683

In [ ]:
model_name = 'tfifg_svd_5folds'

results_dir = './results/'
np.save(results_dir + 'train_' + model_name +'.npy', y_oof.target.values)
sample_sub = pd.read_table(data_dir+'mlboot_test.tsv')
sub = x_te[['uid','target']].copy()
sub['target'] = y_pred
sub.columns = ['cuid','target']
sample_sub = sample_sub.merge(sub, on='cuid', how='left')
np.save(results_dir + 'test_' + model_name +'.npy', sample_sub.target.values)
print('isnull?',sample_sub.target.isnull().any())
sample_sub[['target']].to_csv(results_dir + model_name + '.csv', header=False, index=False)
sample_sub.head()